Authors:
Rajeswari Nakka, G.V.S.N.R.V.Prasad, R.Kiran Kumar

## Importing Libraries

In [4]:
import pandas as pd
import numpy as np
import math
import heapq
import joblib
from sklearn.model_selection import train_test_split

## Loading Data

In [5]:
df = pd.read_csv('/content/drive/MyDrive/CF Project/Datasets/Final 3 Datasets (Full Length)/raw-data_interaction.csv')
df = df.rename(columns={'user_id':'userId', 'recipe_id':'itemId', 'rating': 'Rating'})

d = (dict(enumerate(df['userId'].unique())))
new_d = {}
new_d = {value:key for key, value in d.items()}
df['userId'] = df['userId'].map(new_d)

d = (dict(enumerate(df['itemId'].unique())))
new_d = {}
new_d = {value:key for key, value in d.items()}
df['itemId'] = df['itemId'].map(new_d)

train, test = train_test_split(df[(df['userId']<1500) & (df['itemId']<800)], test_size=0.2)

In [6]:
train_dataframe = train
train_dataframe

,userId,itemId,Rating,dateLastModified
589,585,16,2,2005-10-28T18:00:47.2\n
8525,964,98,5,2011-03-11T04:45:02.693\n
714,708,16,3,2010-06-07T15:18:13.107\n
901,891,21,5,2017-12-26T02:00:41.14\n
3501,720,39,5,2012-06-28T23:16:22.783\n
...,...,...,...,...
20057,1000,179,4,2011-11-12T00:48:22.863\n
339,337,8,5,2014-10-29T17:46:55.66\n
57054,765,542,5,2017-05-16T23:43:59.703\n
10559,1182,105,5,2004-10-04T19:21:29.59\n


In [7]:
test_dataframe = test
test_dataframe

,userId,itemId,Rating,dateLastModified
71199,254,730,4,2010-04-12T17:10:23.83\n
57155,175,543,4,2006-03-12T16:08:06.59\n
20,20,1,5,2016-11-29T22:54:43.717\n
66287,912,621,4,2008-01-25T11:31:20.077\n
804,798,17,3,2015-12-21T18:08:27.623\n
...,...,...,...,...
63848,148,593,5,2017-11-26T18:11:20.843\n
782,776,17,3,2014-08-17T07:31:25.473\n
1075,1059,23,4,2012-03-05T13:06:12.083\n
71138,789,730,5,2010-07-27T23:11:06.797\n


TODO: Split the data in test and train

In [8]:
'''
CREATE PIVOT TABLE
'''
pivot_table = train_dataframe.pivot_table(values='Rating',columns='itemId',index='userId')
# REPLACE NULL VALUES WITH ZEROES
pivot_table_nn = pivot_table.fillna(0)
pivot_table_nn

itemId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,32,33,34,35,37,38,39,40,43,45,47,...,736,737,738,740,741,745,747,750,751,752,753,754,755,756,758,759,762,763,764,765,767,768,769,770,772,776,777,780,781,783,787,789,790,791,792,793,794,795,796,798
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Algorithm** Adaptive Similarity Measure Model
</br>
Input: user-item rating matrix</br>
Output: user-movie rating matrix with predicted ratings</br>

In [9]:
''' 
FOR ALL ITEMS IN THE DATASET M
COMPUTE THE INVERSE USER FREQUENCY(IUF) FOR EACH ITEM m in M
'''
iuf_dict = {} # will store the item ids as keys and values as iuf value caculated
for item in pivot_table_nn.columns:
  item_ratings = pivot_table_nn[item]
  filtered_item_ratings = [x for x in item_ratings if x > 0.0]
  Um = len(filtered_item_ratings)
  U = len(item_ratings)
  iuf = math.log(Um/U)
  iuf_dict[item] = iuf

SHould i drop 0s when finidng similarities in all of the similarity functions

In [10]:
'''
FUCTION TO CALCULATE SIMILARITY_ASM
'''
def sim_asm(user_id1, user_id2):
  avg_u1_rating = np.array(pivot_table_nn.loc[user_id1]).mean()
  avg_u2_rating = np.array(pivot_table_nn.loc[user_id2]).mean()
  numerator = 0.0
  denominator = 0.0
  for item in pivot_table_nn.columns:
    iuf = iuf_dict[item]
    num_subterm1 = pivot_table_nn[item][user_id1] - avg_u1_rating
    num_subterm2 = pivot_table_nn[item][user_id2] - avg_u2_rating
    numerator += ((iuf**2) * num_subterm1 * num_subterm2)
  denominator = abs(numerator)**0.5
  print('numerator,denominator',numerator,denominator)
  similarity = numerator/denominator
  return similarity


call above and below function for each pair of user

In [11]:
'''
FUNCTION TO CALCULATE JACCARD SIMILARITY
'''
def sim_jaccard(user_id1, user_id2):
  numerator=0
  denominator=0
  for item in pivot_table_nn.columns:
    if pivot_table_nn[item][user_id1]>0 and pivot_table_nn[item][user_id2]>0:
      #if ratings for both users available for given movie increase numerator by 1
      numerator += 1 
    if pivot_table_nn[item][user_id1]>0 or pivot_table_nn[item][user_id2]>0:
      #if ratings for both users available for given movie increase numerator by 1
      denominator += 1 
  similarity = 1 - (numerator/denominator)
  return similarity

In [12]:
'''
FUNCTION TO CALCULATE MODIFIED SIMILARITY
'''
def modified_sim(user_id1, user_id2):
  term1 = sim_asm(user_id1, user_id2)
  term2 = sim_jaccard(user_id1, user_id2)
  print(term1,term2)
  modified_simialrity = term1 * term2
  print(modified_simialrity)
  return modified_simialrity

In [22]:

#DICTIONARY
i=0
dict_users = {}
for user in pivot_table_nn.index:
  dict_users[user] = i
  i+=1

In [ ]:
'''
CREATE A SIMILARITY MATRIX OF DIMENSION (NO. OF USERS *  NO. OF USERS)
'''


similarity_matrix = np.zeros([pivot_table_nn.shape[0],pivot_table_nn.shape[0]])
i=0
for user1 in pivot_table_nn.index:
  for user2 in pivot_table_nn.index:
    i+=1
    print("Cell Number:",i)
    if  similarity_matrix[dict_users[user1]][dict_users[user2]] == 0:
      similarity_matrix[dict_users[user1]][dict_users[user2]] = modified_sim(user1,user2)
      similarity_matrix[dict_users[user2]][dict_users[user1]] = similarity_matrix[dict_users[user1]][dict_users[user2]] 
similarity_matrix

Streaming output truncated to the last 5000 lines.
Cell Number: 1634178
Cell Number: 1634179
Cell Number: 1634180
Cell Number: 1634181
Cell Number: 1634182
Cell Number: 1634183
Cell Number: 1634184
Cell Number: 1634185
Cell Number: 1634186
Cell Number: 1634187
Cell Number: 1634188
Cell Number: 1634189
Cell Number: 1634190
Cell Number: 1634191
Cell Number: 1634192
Cell Number: 1634193
Cell Number: 1634194
Cell Number: 1634195
Cell Number: 1634196
Cell Number: 1634197
Cell Number: 1634198
Cell Number: 1634199
Cell Number: 1634200
Cell Number: 1634201
Cell Number: 1634202
Cell Number: 1634203
Cell Number: 1634204
Cell Number: 1634205
Cell Number: 1634206
Cell Number: 1634207
Cell Number: 1634208
Cell Number: 1634209
Cell Number: 1634210
Cell Number: 1634211
Cell Number: 1634212
Cell Number: 1634213
Cell Number: 1634214
Cell Number: 1634215
Cell Number: 1634216
Cell Number: 1634217
Cell Number: 1634218
Cell Number: 1634219
Cell Number: 1634220
Cell Number: 1634221
Cell Number: 1634222
Cell

array([[ 0.        , -1.18597567, -1.18597567, ..., -0.54958355,
         0.8011026 ,  0.71652795],
       [-1.18597567,  0.        ,  0.        , ..., -1.55411659,
        -0.56462074, -0.50501214],
       [-1.18597567,  0.        ,  0.        , ..., -1.55411659,
        -0.56462074, -0.50501214],
       ...,
       [-0.54958355, -1.55411659, -1.55411659, ...,  0.        ,
         6.81922188,  6.09929747],
       [ 0.8011026 , -0.56462074, -0.56462074, ...,  6.81922188,
         0.        ,  0.        ],
       [ 0.71652795, -0.50501214, -0.50501214, ...,  6.09929747,
         0.        ,  0.        ]])

In [ ]:
#CREATE DATAFRAME
similarity_matrix_df = pd.DataFrame(similarity_matrix)
# similarity_matrix_df['original_user_id'] = list(dict_users.keys())
similarity_matrix_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,0.000000,-1.185976,-1.185976,0.585186,-0.454874,-0.942435,0.063119,0.070569,-1.775226,-1.337808,0.070569,-0.764125,0.070569,0.070569,-1.016906,0.070569,0.070569,0.070569,0.070569,0.063119,0.070569,0.070569,0.070569,-1.490130,0.070569,0.070569,-0.825813,-1.523995,-1.229167,-0.623035,-1.685496,-0.482601,-0.623035,-0.623035,-0.623035,-0.623035,-0.557260,0.317142,0.799907,0.715458,...,0.506662,0.801103,1.066816,0.716528,0.801103,-0.770349,0.801103,-0.297464,0.624531,0.801103,0.716528,0.801103,0.716528,0.801103,0.801103,0.897184,0.538339,0.801103,0.716528,0.801103,0.801103,0.801103,0.801103,0.335088,0.801103,0.801103,0.801103,0.803585,0.716528,0.801103,-1.594091,0.687207,0.801103,0.801103,-0.647316,0.716528,0.801103,-0.549584,0.801103,0.716528
1,-1.185976,0.000000,0.000000,-1.001171,-1.458778,-1.891496,-0.874337,-0.977539,-2.376739,-2.161346,-0.977539,-1.582723,-0.977539,-0.977539,-1.603433,-0.977539,-0.977539,-0.977539,-0.977539,-0.874337,-0.977539,-0.977539,-0.977539,-2.258799,-0.977539,-0.977539,-2.300579,-2.060021,-1.852559,-1.161350,-2.933303,-0.899578,-1.161350,-1.161350,-1.161350,-1.161350,-1.038743,-1.934339,-0.566314,-0.506526,...,-0.357098,-0.564621,-1.165918,-0.505012,-0.564621,-2.059993,-0.564621,-1.024230,-1.070915,-0.564621,-0.505012,-0.564621,-0.505012,-0.564621,-0.564621,-0.961287,-1.485817,-0.564621,-0.505012,-0.564621,-0.564621,-0.564621,-0.564621,-1.931311,-0.564621,-0.564621,-0.564621,-1.040799,-0.505012,-0.564621,-2.449316,-1.121052,-0.564621,-0.564621,-1.465616,-0.505012,-0.564621,-1.554117,-0.564621,-0.505012
2,-1.185976,0.000000,0.000000,-1.001171,-1.458778,-1.891496,-0.874337,-0.977539,-2.376739,-2.161346,-0.977539,-1.582723,-0.977539,-0.977539,-1.603433,-0.977539,-0.977539,-0.977539,-0.977539,-0.874337,-0.977539,-0.977539,-0.977539,-2.258799,-0.977539,-0.977539,-2.300579,-2.060021,-1.852559,-1.161350,-2.933303,-0.899578,-1.161350,-1.161350,-1.161350,-1.161350,-1.038743,-1.934339,-0.566314,-0.506526,...,-0.357098,-0.564621,-1.165918,-0.505012,-0.564621,-2.059993,-0.564621,-1.024230,-1.070915,-0.564621,-0.505012,-0.564621,-0.505012,-0.564621,-0.564621,-0.961287,-1.485817,-0.564621,-0.505012,-0.564621,-0.564621,-0.564621,-0.564621,-1.931311,-0.564621,-0.564621,-0.564621,-1.040799,-0.505012,-0.564621,-2.449316,-1.121052,-0.564621,-0.564621,-1.465616,-0.505012,-0.564621,-1.554117,-0.564621,-0.505012
3,0.585186,-1.001171,-1.001171,0.000000,8.609302,9.680227,5.763680,6.443990,-1.377785,9.649124,6.443990,8.595669,6.443990,6.443990,0.314894,6.443990,6.443990,6.443990,6.443990,5.763680,6.443990,6.443990,6.443990,9.631530,6.443990,6.443990,10.827312,-1.148027,-0.426115,0.650667,14.235466,0.504005,0.650667,0.650667,0.650667,0.650667,0.581974,13.068880,1.364771,1.220689,...,0.863778,1.365753,2.026584,1.221567,1.365753,1.686139,1.365753,0.918113,1.446703,1.365753,1.221567,1.365753,1.221567,1.365753,1.365753,1.693273,1.708636,1.365753,1.221567,1.365753,1.365753,1.365753,1.365753,2.006100,1.365753,1.365753,1.365753,1.626132,1.221567,1.365753,-0.277601,1.549649,1.365753,1.365753,1.074072,1.221567,1.365753,1.305420,1.365753,1.221567
4,-0.454874,-1.458778,-1.458778,8.609302,0.000000,15.241014,9.094071,10.167480,-2.126439,15.210238,10.167480,13.548562,10.167480,10.167480,-0.985966,10.167480,10.167480,10.167480,10.167480,9.094071,10.167480,10.167480,10.167480,15.194300,10.167480,10.167480,16.970970,-1.809706,-1.285154,-0.302165,-1.253610,-0.234056,-0.302165,-0.302165,-0.302165,-0.302165,-0.270264,1.715030,1.401693,1.253712,...,0.887372,1.403058,2.014275,1.254934,1.403058,1.190078,1.403058,0.712792,1.369725,1.403058,1.254934,1.403058,1.254934,1.403058,1.403058,1.686107,1.536466,1.403058,1.254934,1.403058,1.403058,1.4

In [ ]:
###drop duplicates? - must drop from dictionary then too?

In [ ]:
# filename = '/content/drive/MyDrive/CF Project/Datasets/similarity_matrix_df_finefood.sav'
# joblib.dump(similarity_matrix_df, filename)


['/content/drive/MyDrive/CF Project/Datasets/similarity_matrix_df_finefood.sav']

In [3]:
import joblib
filename = '/content/drive/MyDrive/CF Project/Datasets/similarity_matrix_df_finefood.sav'
similarity_matrix_df = joblib.load(filename)
similarity_matrix_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249,1250,1251,1252,1253,1254,1255,1256,1257,1258,1259,1260,1261,1262,1263,1264,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,0.000000,-1.185976,-1.185976,0.585186,-0.454874,-0.942435,0.063119,0.070569,-1.775226,-1.337808,0.070569,-0.764125,0.070569,0.070569,-1.016906,0.070569,0.070569,0.070569,0.070569,0.063119,0.070569,0.070569,0.070569,-1.490130,0.070569,0.070569,-0.825813,-1.523995,-1.229167,-0.623035,-1.685496,-0.482601,-0.623035,-0.623035,-0.623035,-0.623035,-0.557260,0.317142,0.799907,0.715458,...,0.506662,0.801103,1.066816,0.716528,0.801103,-0.770349,0.801103,-0.297464,0.624531,0.801103,0.716528,0.801103,0.716528,0.801103,0.801103,0.897184,0.538339,0.801103,0.716528,0.801103,0.801103,0.801103,0.801103,0.335088,0.801103,0.801103,0.801103,0.803585,0.716528,0.801103,-1.594091,0.687207,0.801103,0.801103,-0.647316,0.716528,0.801103,-0.549584,0.801103,0.716528
1,-1.185976,0.000000,0.000000,-1.001171,-1.458778,-1.891496,-0.874337,-0.977539,-2.376739,-2.161346,-0.977539,-1.582723,-0.977539,-0.977539,-1.603433,-0.977539,-0.977539,-0.977539,-0.977539,-0.874337,-0.977539,-0.977539,-0.977539,-2.258799,-0.977539,-0.977539,-2.300579,-2.060021,-1.852559,-1.161350,-2.933303,-0.899578,-1.161350,-1.161350,-1.161350,-1.161350,-1.038743,-1.934339,-0.566314,-0.506526,...,-0.357098,-0.564621,-1.165918,-0.505012,-0.564621,-2.059993,-0.564621,-1.024230,-1.070915,-0.564621,-0.505012,-0.564621,-0.505012,-0.564621,-0.564621,-0.961287,-1.485817,-0.564621,-0.505012,-0.564621,-0.564621,-0.564621,-0.564621,-1.931311,-0.564621,-0.564621,-0.564621,-1.040799,-0.505012,-0.564621,-2.449316,-1.121052,-0.564621,-0.564621,-1.465616,-0.505012,-0.564621,-1.554117,-0.564621,-0.505012
2,-1.185976,0.000000,0.000000,-1.001171,-1.458778,-1.891496,-0.874337,-0.977539,-2.376739,-2.161346,-0.977539,-1.582723,-0.977539,-0.977539,-1.603433,-0.977539,-0.977539,-0.977539,-0.977539,-0.874337,-0.977539,-0.977539,-0.977539,-2.258799,-0.977539,-0.977539,-2.300579,-2.060021,-1.852559,-1.161350,-2.933303,-0.899578,-1.161350,-1.161350,-1.161350,-1.161350,-1.038743,-1.934339,-0.566314,-0.506526,...,-0.357098,-0.564621,-1.165918,-0.505012,-0.564621,-2.059993,-0.564621,-1.024230,-1.070915,-0.564621,-0.505012,-0.564621,-0.505012,-0.564621,-0.564621,-0.961287,-1.485817,-0.564621,-0.505012,-0.564621,-0.564621,-0.564621,-0.564621,-1.931311,-0.564621,-0.564621,-0.564621,-1.040799,-0.505012,-0.564621,-2.449316,-1.121052,-0.564621,-0.564621,-1.465616,-0.505012,-0.564621,-1.554117,-0.564621,-0.505012
3,0.585186,-1.001171,-1.001171,0.000000,8.609302,9.680227,5.763680,6.443990,-1.377785,9.649124,6.443990,8.595669,6.443990,6.443990,0.314894,6.443990,6.443990,6.443990,6.443990,5.763680,6.443990,6.443990,6.443990,9.631530,6.443990,6.443990,10.827312,-1.148027,-0.426115,0.650667,14.235466,0.504005,0.650667,0.650667,0.650667,0.650667,0.581974,13.068880,1.364771,1.220689,...,0.863778,1.365753,2.026584,1.221567,1.365753,1.686139,1.365753,0.918113,1.446703,1.365753,1.221567,1.365753,1.221567,1.365753,1.365753,1.693273,1.708636,1.365753,1.221567,1.365753,1.365753,1.365753,1.365753,2.006100,1.365753,1.365753,1.365753,1.626132,1.221567,1.365753,-0.277601,1.549649,1.365753,1.365753,1.074072,1.221567,1.365753,1.305420,1.365753,1.221567
4,-0.454874,-1.458778,-1.458778,8.609302,0.000000,15.241014,9.094071,10.167480,-2.126439,15.210238,10.167480,13.548562,10.167480,10.167480,-0.985966,10.167480,10.167480,10.167480,10.167480,9.094071,10.167480,10.167480,10.167480,15.194300,10.167480,10.167480,16.970970,-1.809706,-1.285154,-0.302165,-1.253610,-0.234056,-0.302165,-0.302165,-0.302165,-0.302165,-0.270264,1.715030,1.401693,1.253712,...,0.887372,1.403058,2.014275,1.254934,1.403058,1.190078,1.403058,0.712792,1.369725,1.403058,1.254934,1.403058,1.254934,1.403058,1.403058,1.686107,1.536466,1.403058,1.254934,1.403058,1.403058,1.4

In [45]:
def find_delu(mu, user):
  # FIND DEL_U
  count_items_rated_by_user = len([x for x in pivot_table_nn.loc[user] if x > 0])
  term1 = 1 / count_items_rated_by_user
  summation = 0
  for r in pivot_table_nn.loc[user]:
    if r > 0:
      summation += (r-mu)
  del_u = term1 * summation
  return del_u
def find_deli(mu, user, del_u):
  # FIND DEL_I
  count_users_rated_item = len([x for x in pivot_table_nn[item]])
  term1 = 1 / count_users_rated_item
  summation = 0
  for r in pivot_table_nn[item]:
    if r > 0:
      summation += (r - del_u - mu)
  del_i = summation
  return del_i
def find_delui(mu,user):
  # FIND DEL_UI
  del_u = find_delu(mu,user)
  del_i = find_deli(mu,user,del_u)
  del_ui = mu + del_u + del_i
  return del_ui
def find_key(val, my_dict):
  for key, value in my_dict.items():
    if val == value:
      return key
  return "key doesn't exist"

In [67]:
'''
PREDICT THE RATINGS
'''
# FIND MU, GLOBAL AVERAGE
mu = train_dataframe['Rating'].mean()
N = 5 # NUMBER OF NEIGHBOURS TO TAKE
mae = []
pred = []
actuals = []
for index,row in test_dataframe.iterrows():
  user = row['userId']
  item = row['itemId']
  given_rating = row['Rating']
  #GET NEIGHBOUR 'N' USERS
  if user not in dict_users.keys() or item not in pivot_table_nn.columns or dict_users[user] not in similarity_matrix_df.index:
    #cold start problem
    continue
  the_row = similarity_matrix_df[dict_users[user]]
  the_row = the_row.sort_values(ascending=False)
  # print('the_row',the_row[:N])
  neighbour_index = the_row.index
  neighbour_similarities = list(the_row)
  
  # FIND PRED_UI
  num = 0# how to the user_ids here
  deno = 0# how to get 
  # iterate over neighbour users
  the_delui = find_delui(mu,user)
  for i in range(N):
    deno += abs(neighbour_similarities[i])
    real_neighbour_id = find_key(neighbour_index[i],dict_users)
    rating_neigh = pivot_table_nn[item][real_neighbour_id]
    delui_neigh = find_delui(mu,real_neighbour_id)
    num += ((neighbour_similarities[i]) *(rating_neigh-delui_neigh))
  if deno>0:
    term2 = num/deno
  else:
    term2 = 0
  pred_ui = the_delui + term2
  mae.append(abs(pred_ui-given_rating))
  pred.append(pred_ui)
  actuals.append(given_rating)

# print("MAE:",np.array(mae).mean())
# print("Prediction:",np.array(pred))
# print("Actual:",np.array(actuals))

normalized = [((x-min(pred)) / (max(pred) - min(pred)) * 5)+3 for x in pred]
print(np.mean(abs(np.array(normalized)-np.array(actuals))))



0.744742798799487


In [81]:
'''
PREDICT THE RATINGS
'''
# FIND MU, GLOBAL AVERAGE
mu = train_dataframe['Rating'].mean()
N = 5 # NUMBER OF NEIGHBOURS TO TAKE
mae = []
pred = []
actuals = []
for index,row in test_dataframe.iterrows():
  user = row['userId']
  item = row['itemId']
  given_rating = row['Rating']
  #GET NEIGHBOUR 'N' USERS
  if user not in dict_users.keys() or item not in pivot_table_nn.columns or dict_users[user] not in similarity_matrix_df.index:
    #cold start problem
    continue
  the_row = similarity_matrix_df[dict_users[user]]
  the_row = the_row.sort_values(ascending=False)
  # print('the_row',the_row[:N])
  neighbour_index = the_row.index
  neighbour_similarities = list(the_row)
  
  # FIND PRED_UI
  num = 0# how to the user_ids here
  deno = 0# how to get 
  # iterate over neighbour users
  the_delui = find_delui(mu,user)
  for i in range(N):
    deno += abs(neighbour_similarities[i])
    real_neighbour_id = find_key(neighbour_index[i],dict_users)
    rating_neigh = pivot_table_nn[item][real_neighbour_id]
    delui_neigh = find_delui(mu,real_neighbour_id)
    num += ((neighbour_similarities[i]) *(rating_neigh-delui_neigh))
  if deno>0:
    term2 = num/deno
  else:
    term2 = 0
  pred_ui = the_delui + term2
  mae.append(abs(pred_ui-given_rating))
  pred.append(pred_ui)
  actuals.append(given_rating)

# print("MAE:",np.array(mae).mean())
# print("Prediction:",np.array(pred))
# print("Actual:",np.array(actuals))

normalized = [((x-min(pred))/(max(pred)-min(pred))*5)+3 for x in pred]
print("MAE",np.mean(abs(np.array(normalized)-np.array(actuals))))



MAE 0.744742798799487
